In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import time
from datetime import datetime

In [2]:
#Extract data from CSV
df1=pd.read_csv("database.csv")

In [3]:
epoch = datetime(1970, 1, 1)

def mapdateTotime(x):
    try:
        dt = datetime.strptime(x, "%m/%d/%Y")
    except ValueError:
        dt = datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ")
    diff = dt - epoch
    return diff.total_seconds()

df1.Date = df1.Date.apply(mapdateTotime)

In [4]:
col1 = df1[['Date','Latitude','Longitude','Depth']]
col2 = df1['Magnitude']
#Convert to Numpy array
InputX1 = col1.values
InputY1 = col2.values
print(InputX1)

[[ -1.57680000e+08   1.92460000e+01   1.45616000e+02   1.31600000e+02]
 [ -1.57507200e+08   1.86300000e+00   1.27352000e+02   8.00000000e+01]
 [ -1.57420800e+08  -2.05790000e+01  -1.73972000e+02   2.00000000e+01]
 ..., 
 [  1.48288320e+09   3.69179000e+01   1.40426200e+02   1.00000000e+01]
 [  1.48296960e+09  -9.02830000e+00   1.18663900e+02   7.90000000e+01]
 [  1.48305600e+09   3.73973000e+01   1.41410300e+02   1.19400000e+01]]


In [5]:
#Min-max Normalization
X1_min = np.amin(InputX1,0)     
X1_max = np.amax(InputX1,0)   
print("Mininum values:",X1_min)
print("Maximum values:",X1_max)
Y1_min = np.amin(InputY1)     
Y1_max = np.amax(InputY1) 
InputX1_norm = (InputX1-X1_min)/(X1_max-X1_min)
InputY1_norm = InputY1  #No normalization in output

#Reshape
Xfeatures = 3 #Number of input features
Yfeatures = 1 #Number of input features
samples = 23000 # Number of samples

InputX1_reshape = np.resize(InputX1_norm,(samples,Xfeatures))
InputY1_reshape = np.resize(InputY1_norm,(samples,Yfeatures))

Mininum values: [ -1.57680000e+08  -7.70800000e+01  -1.79997000e+02  -1.10000000e+00]
Maximum values: [  1.48305600e+09   8.60050000e+01   1.79998000e+02   7.00000000e+02]


In [6]:
#Training data
batch_size = 2000
InputX1train = InputX1_reshape[0:batch_size,:]
InputY1train = InputY1_reshape[0:batch_size,:]
#Validation data
v_size = 2500
InputX1v = InputX1_reshape[batch_size:batch_size+v_size,:]
InputY1v = InputY1_reshape[batch_size:batch_size+v_size,:]

In [7]:
learning_rate = 0.001
training_iterations = 1000
display_iterations = 200

In [8]:
#Input
X = tf.placeholder(tf.float32,shape=(None,Xfeatures))
#Output
Y = tf.placeholder(tf.float32)

In [9]:
#Neurons
L1 = 3
L2 = 3
L3 = 3

#Layer1 weights
W_fc1 = tf.Variable(tf.random_uniform([Xfeatures,L1]))
b_fc1 = tf.Variable(tf.constant(0.1,shape=[L1]))

#Layer2 weights
W_fc2 = tf.Variable(tf.random_uniform([L1,L2]))
b_fc2 = tf.Variable(tf.constant(0.1,shape=[L2]))

#Layer3 weights
W_fc3 = tf.Variable(tf.random_uniform([L2,L3]))
b_fc3 = tf.Variable(tf.constant(0.1,shape=[L3]))

#Output layer weights
W_fO= tf.Variable(tf.random_uniform([L3,Yfeatures]))
b_fO = tf.Variable(tf.constant(0.1,shape=[Yfeatures]))

In [10]:
#Layer 1
matmul_fc1=tf.matmul(X, W_fc1) + b_fc1
h_fc1 = tf.nn.relu(matmul_fc1)   #ReLU activation
#Layer 2
matmul_fc2=tf.matmul(h_fc1, W_fc2) + b_fc2
h_fc2 = tf.nn.relu(matmul_fc2)   #ReLU activation
#Layer 3
matmul_fc3=tf.matmul(h_fc2, W_fc3) + b_fc3
h_fc3 = tf.nn.relu(matmul_fc3)   #ReLU activation
#Output layer
matmul_fc4=tf.matmul(h_fc3, W_fO) + b_fO
output_layer = matmul_fc4  #linear activation

In [11]:
#Loss function
mean_square =  tf.reduce_mean(tf.square(Y-output_layer))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(mean_square)

#Operation to save variables
saver = tf.train.Saver()

In [12]:
#Initialization and session
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print("Training loss:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}))
    for i in range(training_iterations):
        sess.run([train_step],feed_dict={X:InputX1train,Y:InputY1train})
        if i%display_iterations ==0:
            print("Training loss is:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}),"at itertion:",i)
            print("Validation loss is:",sess.run([mean_square],feed_dict={X:InputX1v,Y:InputY1v}),"at itertion:",i)
    # Save the variables to disk.
    save_path = saver.save(sess, "/tmp/earthquake_model.ckpt")
    print("Model saved in file: %s" % save_path)

    print("Final training loss:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}))
    print("Final validation loss:",sess.run([mean_square],feed_dict={X:InputX1v,Y:InputY1v}))

Training loss: [7.8162642]
Training loss is: [7.6603436] at itertion: 0
Validation loss is: [6.4701533] at itertion: 0
Training loss is: [1.7573606] at itertion: 200
Validation loss is: [1.680984] at itertion: 200
Training loss is: [1.09281] at itertion: 400
Validation loss is: [1.0900164] at itertion: 400
Training loss is: [0.69149578] at itertion: 600
Validation loss is: [0.70860344] at itertion: 600
Training loss is: [0.45098665] at itertion: 800
Validation loss is: [0.46836597] at itertion: 800
Model saved in file: /tmp/earthquake_model.ckpt
Final training loss: [0.31291121]
Final validation loss: [0.32535991]


In [13]:
#Testing
lat = input("Enter Latitude between -77 to 86:")
long = input("Enter Longitude between -180 to 180:")
depth = input("Enter Depth between 0 to 700:")
date = input("Enter the date (Month/Day/Year format):")
InputX2 = np.asarray([[lat,long,depth,mapdateTotime(date)]],dtype=np.float32)
InputX2_norm = (InputX2-X1_min)/(X1_max-X1_min)
InputX1test = np.resize(InputX2_norm,(1,Xfeatures))
with tf.Session() as sess:
    # Restore variables from disk for validation.
    saver.restore(sess, "/tmp/earthquake_model.ckpt")
    print("Model restored.")
    #print("Final validation loss:",sess.run([mean_square],feed_dict={X:InputX1v,Y:InputY1v}))
    print("output:",sess.run([output_layer],feed_dict={X:InputX1test}))

Enter Latitude between -77 to 86:11.938
Enter Longitude between -180 to 180:126.427
Enter Depth between 0 to 700:30
Enter the date (Month/Day/Year format):01/09/1965
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
Model restored.
output: [array([[ 6.10636854]], dtype=float32)]


In [6]:
import numpy as np
import tensorflow as tf
import pandas as pd
import time
from datetime import datetime
#Extract data from CSV
df1=pd.read_csv("database.csv")
epoch = datetime(1970, 1, 1)

def mapdateTotime(x):
    try:
        dt = datetime.strptime(x, "%m/%d/%Y")
    except ValueError:
        dt = datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ")
    diff = dt - epoch
    return diff.total_seconds()

df1.Date = df1.Date.apply(mapdateTotime)
col1 = df1[['Date','Latitude','Longitude','Depth']]
col2 = df1['Magnitude']
#Convert to Numpy array
InputX1 = col1.as_matrix()
InputY1 = col2.as_matrix()
print(InputX1)
#Min-max Normalization
X1_min = np.amin(InputX1,0)     
X1_max = np.amax(InputX1,0)   
print("Mininum values:",X1_min)
print("Maximum values:",X1_max)
Y1_min = np.amin(InputY1)     
Y1_max = np.amax(InputY1) 
InputX1_norm = (InputX1-X1_min)/(X1_max-X1_min)
InputY1_norm = InputY1  #No normalization in output

#Reshape
Xfeatures = 3 #Number of input features
Yfeatures = 1 #Number of input features
samples = 23000 # Number of samples

InputX1_reshape = np.resize(InputX1_norm,(samples,Xfeatures))
InputY1_reshape = np.resize(InputY1_norm,(samples,Yfeatures))
#Training data
batch_size = 2000
InputX1train = InputX1_reshape[0:batch_size,:]
InputY1train = InputY1_reshape[0:batch_size,:]
#Validation data
v_size = 2500
InputX1v = InputX1_reshape[batch_size:batch_size+v_size,:]
InputY1v = InputY1_reshape[batch_size:batch_size+v_size,:]
learning_rate = 0.001
training_iterations = 1000
display_iterations = 200
#Input
X = tf.placeholder(tf.float32,shape=(None,Xfeatures))
#Output
Y = tf.placeholder(tf.float32)
#Neurons
L1 = 3
L2 = 3
L3 = 3

#Layer1 weights
W_fc1 = tf.Variable(tf.random_uniform([Xfeatures,L1]))
b_fc1 = tf.Variable(tf.constant(0.1,shape=[L1]))

#Layer2 weights
W_fc2 = tf.Variable(tf.random_uniform([L1,L2]))
b_fc2 = tf.Variable(tf.constant(0.1,shape=[L2]))

#Layer3 weights
W_fc3 = tf.Variable(tf.random_uniform([L2,L3]))
b_fc3 = tf.Variable(tf.constant(0.1,shape=[L3]))

#Output layer weights
W_fO= tf.Variable(tf.random_uniform([L3,Yfeatures]))
b_fO = tf.Variable(tf.constant(0.1,shape=[Yfeatures]))
#Layer 1
matmul_fc1=tf.matmul(X, W_fc1) + b_fc1
h_fc1 = tf.nn.relu(matmul_fc1)   #ReLU activation
#Layer 2
matmul_fc2=tf.matmul(h_fc1, W_fc2) + b_fc2
h_fc2 = tf.nn.relu(matmul_fc2)   #ReLU activation
#Layer 3
matmul_fc3=tf.matmul(h_fc2, W_fc3) + b_fc3
h_fc3 = tf.nn.relu(matmul_fc3)   #ReLU activation
#Output layer
matmul_fc4=tf.matmul(h_fc3, W_fO) + b_fO
output_layer = matmul_fc4  #linear activation
#Loss function
mean_square =  tf.reduce_mean(tf.square(Y-output_layer))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(mean_square)

#Operation to save variables
saver = tf.train.Saver()
#Initialization and session
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print("Training loss:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}))
    for i in range(training_iterations):
        sess.run([train_step],feed_dict={X:InputX1train,Y:InputY1train})
        if i%display_iterations ==0:
            print("Training loss is:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}),"at itertion:",i)
            print("Validation loss is:",sess.run([mean_square],feed_dict={X:InputX1v,Y:InputY1v}),"at itertion:",i)
    # Save the variables to disk.
    save_path = saver.save(sess, "/tmp/earthquake_model.ckpt")
    print("Model saved in file: %s" % save_path)

    print("Final training loss:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}))
    print("Final validation loss:",sess.run([mean_square],feed_dict={X:InputX1v,Y:InputY1v}))
# Testing
lat = input("Enter Latitude between -77 to 86:")
long = input("Enter Longitude between -180 to 180:")
depth = input("Enter Depth between 0 to 700:")
date = input("Enter the date (Month/Day/Year format):")

InputX2 = np.asarray([[lat, long, depth, mapdateTotime(date)]], dtype=np.float32)
InputX2_norm = (InputX2 - X1_min) / (X1_max - X1_min)
InputX1test = np.resize(InputX2_norm, (1, Xfeatures))

import sys
import os

# Save the current standard output
original_stdout = sys.stdout

# Open a temporary file to redirect the standard output
with open(os.devnull, 'w') as f:
    sys.stdout = f  # Redirect standard output to the temporary file

    # Load the model
    loaded_model = tf.keras.models.load_model("/tmp/earthquake_model.h5")
    print("Model loaded.")

    # Predict
    output = loaded_model.predict(InputX1test)

# Restore the original standard output
sys.stdout = original_stdout

# Print the desired output
print("The magnitude of the possible earthquake is:", output[0][0])



AttributeError: module 'numpy' has no attribute 'typeDict'

In [8]:
import numpy as np
import tensorflow as tf
import pandas as pd
from datetime import datetime

# Extract data from CSV
df1 = pd.read_csv("database.csv")
epoch = datetime(1970, 1, 1)


def mapdateTotime(x):
    try:
        dt = datetime.strptime(x, "%m/%d/%Y")
    except ValueError:
        dt = datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ")
    diff = dt - epoch
    return diff.total_seconds()


df1.Date = df1.Date.apply(mapdateTotime)
col1 = df1[['Date', 'Latitude', 'Longitude', 'Depth']]
col2 = df1['Magnitude']

# Convert to Numpy array
InputX1 = col1.values
InputY1 = col2.values
print(InputX1)

# Min-max Normalization
X1_min = np.amin(InputX1, 0)
X1_max = np.amax(InputX1, 0)
print("Minimum values:", X1_min)
print("Maximum values:", X1_max)
Y1_min = np.amin(InputY1)
Y1_max = np.amax(InputY1)
InputX1_norm = (InputX1 - X1_min) / (X1_max - X1_min)
InputY1_norm = InputY1  # No normalization in output

# Reshape
Xfeatures = 3  # Number of input features
Yfeatures = 1  # Number of input features
samples = 23000  # Number of samples

InputX1_reshape = np.resize(InputX1_norm, (samples, Xfeatures))
InputY1_reshape = np.resize(InputY1_norm, (samples, Yfeatures))

# Training data
batch_size = 2000
InputX1train = InputX1_reshape[0:batch_size, :]
InputY1train = InputY1_reshape[0:batch_size, :]

# Validation data
v_size = 2500
InputX1v = InputX1_reshape[batch_size:batch_size + v_size, :]
InputY1v = InputY1_reshape[batch_size:batch_size + v_size, :]

learning_rate = 0.001
training_iterations = 1000
display_iterations = 200

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(3, activation='relu', input_shape=(Xfeatures,)),
    tf.keras.layers.Dense(3, activation='relu'),
    tf.keras.layers.Dense(3, activation='relu'),
    tf.keras.layers.Dense(Yfeatures)
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss=tf.keras.losses.MeanSquaredError())

# Training
model.fit(InputX1train, InputY1train, batch_size=batch_size, epochs=training_iterations, verbose=1)

# Evaluation
train_loss = model.evaluate(InputX1train, InputY1train, verbose=0)
val_loss = model.evaluate(InputX1v, InputY1v, verbose=0)
print("Training loss:", train_loss)
print("Validation loss:", val_loss)

# Save the model
model.save("/tmp/earthquake_model.h5")
print("Model saved.")

# Testing
lat = input("Enter Latitude between -77 to 86:")
long = input("Enter Longitude between -180 to 180:")
depth = input("Enter Depth between 0 to 700:")
date = input("Enter the date (Month/Day/Year format):")

InputX2 = np.asarray([[lat, long, depth, mapdateTotime(date)]], dtype=np.float32)
InputX2_norm = (InputX2 - X1_min) / (X1_max - X1_min)
InputX1test = np.resize(InputX2_norm, (1, Xfeatures))

# Load the model
loaded_model = tf.keras.models.load_model("/tmp/earthquake_model.h5")
print("Model loaded.")

# Predict
output = loaded_model.predict(InputX1test)
print("Output:", output)


AttributeError: module 'numpy' has no attribute 'typeDict'